Step 1: import required libraries. 

In [1]:
# Python libraries necessary for the proposed project

import os
import wave
import pandas as pd 
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav

from python_speech_features import mfcc
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'pyaudio'

Step 2: Load dataset

In [11]:
# Instantiate variables

# New user's voiceprint input enrollment for authentication stored wav format.
voiceprint1 = 'new_user_voice.wav'

# Dataset of voiceprints extracted from database.
voice_data = pd.read_csv('./data/common_voice_export.csv')

# Assume we have a copy of the username 'John Doe' voiceprint on record that we can use for authentication knowing he/she have not logged inn for over a year.
voiceprint2 = "new_user.wav"; # Replace this with path to a specific dataset file (wav file).    

voice_data.head()


,chroma,centroid,bandwidth,zcr,energy,contrast,rolloff,pitch,mfcc_1,mfcc_2,...,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,label
0,0.373881,2235.102203,1849.930437,0.062523,1425.328613,17.255016,4055.032169,1102.372528,-441.898895,70.120132,...,-0.040477,-10.635201,-2.460746,-13.406237,-11.991928,3.680081,-5.305139,-7.221350,2.721159,male
1,0.481633,3869.845830,3876.311939,0.109316,1370.310791,22.078891,7707.965598,339.277457,-389.820282,95.328094,...,23.752153,14.203654,-22.785452,6.609687,-1.138791,-7.238389,-4.671542,4.251569,2.941746,male
2,0.450715,2911.612358,3516.134592,0.064602,2303.223145,22.599502,6363.718517,226.289380,-360.531097,100.840355,...,-14.563050,13.379120,0.349416,3.240715,8.958460,-14.926766,-15.971193,-3.133084,-4.107062,male
3,0.426196,2811.887759,2812.845766,0.077350,6169.613281,22.838346,5653.200344,245.272685,-334.961639,124.599457,...,-12.784635,15.460780,-10.657638,-5.839045,-5.830469,-17.864231,2.040766,-11.533746,-10.315263,male
4,0.492532,2478.568114,2411.059334,0.067513,1180.585938,20.290361,4971.099934,392.618866,-423.410004,84.552406,...,-4.241421,9.290192,2.203606,-1.573582,4.506608,-13.449237,0.861690,-1.008762,-2.143651,male


Step 3: Enroll user's voice input for authentication.

In [ ]:
# User enrollment
FRAMES_PER_BUFFER = 3200
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000

p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=FRAMES_PER_BUFFER
)

print("Start Recording...")

seconds = 5
frames = []
for i in range(0, int(RATE / FRAMES_PER_BUFFER * seconds)):
    data = stream.read(FRAMES_PER_BUFFER)
    frames.append(data)

stream.stop_stream()
stream.close()
p.terminate()

obj = wave.open("new_user.wav", "wb")
obj.setnchannels(CHANNELS)
obj.setsampwidth(p.get_sample_size(FORMAT))
obj.setframerate(RATE)
obj.writeframes(b"".join(frames))

print("Stop Recording...")

obj.close()

Step 4: Voiceprint matching

In [ ]:
# Function to extract unique features of a personv
def extract_mfcc_features(wav_file, num_cepstrum=13):
    rate, signal = wav.read(wav_file)
    mfcc_features = mfcc(signal, rate, numcep=num_cepstrum)
    # print("MFCC Features:", mfcc_features)
    return mfcc_features


# Function to authentiicate user with copy of voiceprint in record or to store it in database. 
def compare_voiceprints(voiceprint1, voiceprint2):
    features1 = extract_mfcc_features(voiceprint1)
    features2 = extract_mfcc_features(voiceprint2)

    # Ensure both features have the same length
    min_len = min(features1.shape[0], features2.shape[0])
    features1  = features1[:min_len]
    features2  = features2[:min_len]

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(features1, features2)

    # print("Similarity Matrix:", similarity_matrix)

    # Take the mean similarity score across all frames
    similarity_score = np.mean(similarity_matrix)

    return similarity_score

# voiceprint1 = 'new_user_voice.wav'
# voiceprint2 = 'new_user.wav'

similarity_score = compare_voiceprints(voiceprint1, voiceprint1)
print("Similarity Score:", similarity_score)
